In [1]:
import pandas as pd
import re
def get_Scores(df,phoneName) :
    data =pd.DataFrame(df, columns=['Phone Name','Performance','Display','Battery','Camera'])
    performance_df=data.copy()
    columns_to_drop = list(performance_df.columns)
    performance_df[['Core', 'Processor', 'RAM']] = performance_df['Performance'].str.split('\n', expand=True).iloc[:,:-1]
    performance_df = performance_df.drop(columns_to_drop, axis=1)
    def calculate_phone_score(core, processor, ram):
        # Define a dictionary to assign points based on processors
        processor_scores = {
            'Snapdragon 8 Gen 2':1, 
            'Apple A16 Bionic':8,
           'MediaTek Dimensity 9200':2, 
            'Google Tensor G2':3,
           'MediaTek Dimensity 9000 Plus':2,
            'Apple A15 Bionic':8,
           'Snapdragon 8 Plus Gen 1':5, 
            'Snapdragon 8 Gen 1':4,
           'MediaTek Dimensity 9000':3,
            'MediaTek Dimensity 7050':3,
           'Snapdragon 888':3.5,
            'Snapdragon 888 Plus':4,
            'MediaTek Dimensity 8100':3,
           'MediaTek Dimensity 8100 Max':3.5,
            'Snapdragon 870':3.5,
           'MediaTek Dimensity 920':2,
            'MediaTek Dimensity 1200':2.5,
            # Add more processors and their scores as needed
        }

        # Assign base scores based on core count and RAM size
        core_score = 0
        if 'Octa core' in core:
            core_score = 4
        elif 'Hexa core' in core:
            core_score = 3
        elif 'Quad core' in core:
            core_score = 2
        elif 'Dual core' in core:
            core_score = 1

        ram_score = 0
        if '12 GB' in ram:
            ram_score = 4
        elif '8 GB' in ram:
            ram_score = 3
        elif '6 GB' in ram:
            ram_score = 2
        elif '4 GB' in ram:
            ram_score = 1

        # Get additional processor score from the dictionary
        processor_score = processor_scores.get(processor, 0)

        # Calculate the total score
        total_score = core_score + ram_score + processor_score
        return total_score

    # Apply the function to the DataFrame to calculate the scores
    data['Performance_Score'] = performance_df.apply(lambda row: calculate_phone_score(row['Core'], row['Processor'], row['RAM']), axis=1)
    display_df=data.copy()
    columns_to_drop = list(display_df.columns)
    display_df[['Size', 'Screen_Type', 'Refresh_Rate']]=display_df['Display'].str.split('\n', expand=True).iloc[:,:-1]
    display_df = display_df.drop(columns_to_drop, axis=1)
    def calculate_display_score(size, screen_type, refresh_rate):
        # Score based on screen size (larger screen size gets higher score)
        size_score = float(size.split()[0])

        # Score based on screen type (higher PPI gets higher score, AMOLED screen gets additional points)
        try:
            ppi = float(screen_type.split()[0])
            screen_type_score = ppi / 100  # Scale down PPI to be in a reasonable range
            if 'AMOLED' in screen_type.upper():
                screen_type_score += 5  # Additional points for AMOLED screen
            elif 'OLED' in screen_type.upper():
                screen_type_score += 3     
        except (ValueError, IndexError):
            screen_type_score = 0

        # Score based on refresh rate (higher refresh rate gets higher score)
        refresh_rate_score = 0
        if '90 Hz' in refresh_rate:
            refresh_rate_score = 1
        elif '120 Hz' in refresh_rate:
            refresh_rate_score = 2
        elif '144 Hz' in refresh_rate:
            refresh_rate_score = 3    


        # Calculate the total score
        total_score = size_score + screen_type_score + refresh_rate_score
        return total_score

    # Apply the function to the DataFrame to calculate the scores
    data['Display_Score'] = display_df.apply(lambda row: calculate_display_score(row['Size'], row['Screen_Type'], row['Refresh_Rate']), axis=1)
    battery_df=data.copy()
    columns_to_drop = list(battery_df.columns)
    battery_df[['CapacityMah', 'Charge_Type']]=battery_df['Battery'].str.split('\n', expand=True).iloc[:,:-2]
    battery_df = battery_df.drop(columns_to_drop, axis=1)
    columns_to_drop = list(battery_df.columns)
    battery_df[['Capacity','Unit']]=battery_df['CapacityMah'].str.split(' ',expand=True)
    battery_df = battery_df.drop(columns_to_drop, axis=1)
    battery_df = battery_df.drop(['Unit'], axis=1)
    replace_values = lambda x: 1 if 3000 < x <= 3500 else (2 if 3500 < x <= 4000 else (3 if 4000< x <= 4500 else (4 if 4500 < x <= 5000 else 5)))

    battery_df['Capacity']=battery_df['Capacity'].astype('int')
    data['Battery_Score'] = battery_df['Capacity'].apply(replace_values)
    camera_df=data.copy()
    columns_to_drop = list(camera_df.columns)
    camera_df[['Back','Flash','Front']] =camera_df['Camera'].str.split('\n', expand=True).iloc[:,:-1]
    camera_df = camera_df.drop(columns_to_drop, axis=1)
    camera_df = camera_df.drop(['Flash'], axis=1)
    def sum_of_digits(s):
        # Extract digits using regular expression and sum them up
        digits = [int(digit) for digit in re.findall(r'\d+', s)]
        return sum(digits)

    # Apply the function to each row using apply() along axis=1
    data['Camera_score'] = camera_df.apply(lambda row: sum_of_digits(row['Back'] + row['Front']), axis=1)
    data['Performance_Score']=round((5*data['Performance_Score']/max(data['Performance_Score'])),2)
    data['Display_Score']=round((5*data['Display_Score']/max(data['Display_Score'])),2)
    data['Battery_Score']=round((5*data['Battery_Score']/max(data['Battery_Score'])),2)
    data['Camera_score']=round((5*data['Camera_score']/max(data['Camera_score'])),2)

    data = data.drop(['Performance','Display','Battery','Camera'], axis=1)
    
    Averages=[data['Performance_Score'].mean(),data['Display_Score'].mean(),data['Battery_Score'].mean(),data['Camera_score'].mean()]

    Averages = list(map(lambda x: round(x, 2),Averages))
    def get_phone_info(phone_name):
        row = data[data['Phone Name'] == phone_name].iloc[0]
        phone_info = row.values.tolist()
        return phone_info

    phone_info = get_phone_info(phoneName)
    phone_info=phone_info[1:]
    phoneScore=[]
    phoneScore.append(phone_info)
    phoneScore.append(Averages)
    return phoneScore

In [2]:
df=[['Samsung Galaxy S23 Ultra 5G', 'Octa core (3.36 GHz, Single Core + 2.8 GHz, Quad core + 2 GHz, Tri core)\nSnapdragon 8 Gen 2\n12 GB RAM\n', '6.8 inches (17.27 cm)\n501 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nFast Charging\nUSB Type-C Port\n', '200 + 12 + 10 + 10 MP Quad Primary Cameras\nLED Flash\n12 MP Front Camera\n'], ['Apple iPhone 14 Pro Max', 'Hexa Core (3.46 GHz, Dual Core + 2.02 GHz, Quad core)\nApple A16 Bionic\n6 GB RAM\n', '6.7 inches (17.02 cm)\n460 PPI, OLED\n120 Hz Refresh Rate\n', '4323 mAh\nFast Charging\nLightning Port\n', '48 MP + 12 MP + 12 MP Triple Primary Cameras\nLED Flash\n12 MP Front Camera\n'], ['vivo X90 Pro', 'Octa core (3.05 GHz, Single Core + 2.85 GHz, Tri core + 1.8 GHz, Quad core)\nMediaTek Dimensity 9200\n12 GB RAM\n', '6.78 inches (17.22 cm)\n453 PPI, AMOLED\n120 Hz Refresh Rate\n', '4870 mAh\nFlash Charging\nUSB Type-C Port\n', '50 MP + 12 MP + 50 MP Triple Primary Cameras\nDual-color LED Flash\n32 MP Front Camera\n'], ['Xiaomi 13 Pro', 'Octa core (3.2 GHz, Single Core + 2.8 GHz, Quad core + 2 GHz, Tri core)\nSnapdragon 8 Gen 2\n12 GB RAM\n', '6.73 inches (17.09 cm)\n521 PPI, AMOLED\n120 Hz Refresh Rate\n', '4820 mAh\nFast Charging\nUSB Type-C Port\n', '50 MP + 50 MP + 50 MP Triple Primary Cameras\nDual-color LED Flash\n32 MP Front Camera\n'], ['Google Pixel 7 Pro 5G', 'Octa core (2.85 GHz, Dual Core + 2.35 GHz, Dual core + 1.8 GHz, Quad core)\nGoogle Tensor G2\n12 GB RAM\n', '6.7 inches (17.02 cm)\n513 PPI, OLED\n120 Hz Refresh Rate\n', '5000 mAh\nFast Charging\nUSB Type-C Port\n', '50 MP + 12 MP + 48 MP Triple Primary Cameras\nDual LED Flash\n10.8 MP Front Camera\n'], ['OPPO Find N2 Flip', 'Octa core (3.2 GHz, Single Core + 3 GHz, Tri core + 1.8 GHz, Quad core)\nMediaTek Dimensity 9000 Plus\n8 GB RAM\n', '6.8 inches (17.27 cm)\n403 PPI, AMOLED\n120 Hz Refresh Rate\n', '4300 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '50 MP + 8 MP Dual Primary Cameras\nLED Flash\n32 MP Front Camera\n'], ['Samsung Galaxy S23 Plus 5G', 'Octa core (3.36 GHz, Single Core + 2.8 GHz, Quad core + 2 GHz, Tri core)\nSnapdragon 8 Gen 2\n8 GB RAM\n', '6.6 inches (16.76 cm)\n390 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '4700 mAh\nFast Charging\nUSB Type-C Port\n', '50 MP + 12 MP + 10 MP Triple Primary Cameras\nLED Flash\n12 MP Front Camera\n'], ['Samsung Galaxy S23 5G', 'Octa core (3.36 GHz, Single Core + 2.8 GHz, Quad core + 2 GHz, Tri core)\nSnapdragon 8 Gen 2\n8 GB RAM\n', '6.1 inches (15.49 cm)\n422 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '3900 mAh\nFast Charging\nUSB Type-C Port\n', '50 MP + 12 MP + 10 MP Triple Primary Cameras\nLED Flash\n12 MP Front Camera\n'], ['Apple iPhone 13 Pro Max', 'Hexa Core (3.23 GHz, Dual Core + 1.82 GHz, Quad core)\nApple A15 Bionic\n6 GB RAM\n', '6.7 inches (17.02 cm)\n457 PPI, OLED\n120 Hz Refresh Rate\n', '4352 mAh\nFast Charging\nLightning Port\n', '12 MP + 12 MP + 12 MP Triple Primary Cameras\nDual LED Flash\n12 MP Front Camera\n'], ['Samsung Galaxy Z Fold 4 5G', 'Octa core (3.18 GHz, Single Core + 2.7 GHz, Tri core + 2 GHz, Quad core)\nSnapdragon 8 Plus Gen 1\n12 GB RAM\n', '7.6 inches (19.3 cm)\n373 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '4400 mAh\nFast Charging\nUSB Type-C Port\n', '50 MP + 12 MP + 10 MP Triple Primary Cameras\nLED Flash\n10 MP + 4 MP Dual Front Cameras\n'], ['Samsung Galaxy S22 Ultra', 'Octa core (3 GHz, Single Core + 2.4 GHz, Tri core + 1.7 GHz, Quad core)\nSnapdragon 8 Gen 1\n12 GB RAM\n', '6.8 inches (17.27 cm)\n501 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nFast Charging\nUSB Type-C Port\n', '108 + 12 + 10 + 10 MP Quad Primary Cameras\nLED Flash\n40 MP Front Camera\n'], ['vivo X80 Pro 5G', 'Octa core (3 GHz, Single Core + 2.5 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 8 Gen 1\n12 GB RAM\n', '6.78 inches (17.22 cm)\n518 PPI, AMOLED\n120 Hz Refresh Rate\n', '4700 mAh\nFlash Charging\nUSB Type-C Port\n', '50 + 48 + 12 + 8 MP Quad Primary Cameras\nLED Flash\n32 MP Front Camera\n'], ['OnePlus 11', 'Octa core (3.2 GHz, Single Core + 2.8 GHz, Quad core + 2 GHz, Tri core)\nSnapdragon 8 Gen 2\n8 GB RAM\n', '6.7 inches (17.02 cm)\n526 PPI, Super Fluid AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '50 MP + 48 MP + 32 MP Triple Primary Cameras\nDual LED Flash\n16 MP Front Camera\n'], ['OnePlus Nord 3 5G', 'Octa core (3.05 GHz, Single Core + 2.85 GHz, Tri core + 1.8 GHz, Quad core)\nMediaTek Dimensity 9000\n8 GB RAM\n', '6.74 inches (17.12 cm)\n451 PPI, Super Fluid AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['OPPO Reno10 Pro+ 5G', 'Octa core (3.2 GHz, Single Core + 2.75 GHz, Tri core + 2 GHz, Quad core)\nSnapdragon 8 Plus Gen 1\n12GB RAM + 256GB ROM\n', '6.74 inches (17.12 cm)\n451 PPI, AMOLED\n120Hz LTPS dynamic Refresh Rate\n', '4700 mAh\nSuperVOOC™ Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 64 MP Triple Primary Cameras\nLED Flash\n32MP Front Camera\n'], ['realme 11 Pro Plus', 'Octa core (2.6 GHz, Dual Core + 2 GHz, Hexa Core)\nMediaTek Dimensity 7050\n8 GB RAM\n', '6.7 inches (17.02 cm)\n394 PPI, AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '200 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n32 MP Front Camera\n'], ['Apple iPhone 14 Plus', 'Hexa Core (3.23 GHz, Dual Core + 1.82 GHz, Quad core)\nApple A15 Bionic\n6 GB RAM\n', '6.7 inches (17.02 cm)\n457 PPI, OLED\n60 Hz Refresh Rate\n', '4325 mAh\nFast Charging\nLightning Port\n', '12 MP + 12 MP Dual Primary Cameras\nLED Flash\n12 MP Front Camera\n'], ['Samsung Galaxy Z Flip 4 5G', 'Octa core (3.2 GHz, Single Core + 2.75 GHz, Tri core + 2 GHz, Quad core)\nSnapdragon 8 Plus Gen 1\n8 GB RAM\n', '6.7 inches (17.02 cm)\n426 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '3700 mAh\nFast Charging\nUSB Type-C Port\n', '12 MP + 12 MP Dual Primary Cameras\nLED Flash\n10 MP Front Camera\n'], ['iQOO 11 5G', 'Octa core (3.2 GHz, Single Core + 2.8 GHz, Quad core + 2 GHz, Tri core)\nSnapdragon 8 Gen 2\n8 GB RAM\n', '6.78 inches (17.22 cm)\n518 PPI, AMOLED\n144 Hz Refresh Rate\n', '5000 mAh\nFlash Charging\nUSB Type-C Port\n', '50 MP + 13 MP + 8 MP Triple Primary Cameras\nDual LED Flash\n16 MP Front Camera\n'], ['OnePlus 11R', 'Octa core (3.2 GHz, Single Core + 2.75 GHz, Tri core + 2 GHz, Quad core)\nSnapdragon 8 Plus Gen 1\n8 GB RAM\n', '6.74 inches (17.12 cm)\n451 PPI, Super Fluid AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['Xiaomi 12 Pro 5G', 'Octa core (3 GHz, Single Core + 2.5 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 8 Gen 1\n8 GB RAM\n', '6.73 inches (17.09 cm)\n521 PPI, AMOLED\n120 Hz Refresh Rate\n', '4600 mAh\nHyper Charging 4.0\nUSB Type-C Port\n', '50 MP + 50 MP + 50 MP Triple Primary Cameras\nDual-color LED Flash\n32 MP Front Camera\n'], ['iQOO Neo 7 Pro', 'Octa core (3.19 GHz, Single Core + 2.75 GHz, Tri core + 2 GHz, Quad core)\nSnapdragon 8 Plus Gen 1\n8 GB RAM\n', '6.78 inches (17.22 cm)\n388 PPI, AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nFlash Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['OnePlus 10 Pro', 'Octa core (3 GHz, Single Core + 2.5 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 8 Gen 1\n8 GB RAM\n', '6.7 inches (17.02 cm)\n526 PPI, Fluid AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '48 MP + 50 MP + 8 MP Triple Primary Cameras\nDual LED Flash\n32 MP Front Camera\n'], ['Samsung Galaxy S22 Plus', 'Octa core (3 GHz, Single Core + 2.4 GHz, Tri core + 1.7 GHz, Quad core)\nSnapdragon 8 Gen 1\n8 GB RAM\n', '6.6 inches (16.76 cm)\n390 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nFast Charging\nUSB Type-C Port\n', '50 MP + 12 MP + 10 MP Triple Primary Cameras\nLED Flash\n10 MP Front Camera\n'], ['vivo X80 5G', 'Octa core (3.05 GHz, Single Core + 2.85 GHz, Tri core)\nMediaTek Dimensity 9000\n8 GB RAM\n', '6.78 inches (17.22 cm)\n388 PPI, AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nFast Charging\nUSB Type-C Port\n', '50 MP + 12 MP + 12 MP Triple Primary Cameras\nLED Flash\n32 MP Front Camera\n'], ['Samsung Galaxy Z Fold 3', 'Octa core (2.84 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 888\n12 GB RAM\n', '7.6 inches (19.3 cm)\n372 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '4400 mAh\nFast Charging\nUSB Type-C Port\n', '12 MP + 12 MP + 12 MP Triple Primary Cameras\nLED Flash\n10 MP + 4 MP Dual Front Cameras\n'], ['Samsung Galaxy Z Flip 3', 'Octa core (2.84 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 888\n8 GB RAM\n', '6.7 inches (17.02 cm)\n426 PPI, Dynamic AMOLED\n120 Hz Refresh Rate\n', '3300 mAh\nFast Charging\nUSB Type-C Port\n', '12 MP + 12 MP Dual Primary Cameras\nLED Flash\n10 MP Front Camera\n'], ['iQOO 9 Pro 5G', 'Octa core (3 GHz, Single Core + 2.5 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 8 Gen 1\n8 GB RAM\n', '6.78 inches (17.22 cm)\n518 PPI, AMOLED\n120 Hz Refresh Rate\n', '4700 mAh\nFlash Charging\nUSB Type-C Port\n', '50 MP + 50 MP + 16 MP Triple Primary Cameras\nDual LED Flash\n16 MP Front Camera\n'], ['vivo X70 Pro Plus', 'Octa core (3 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 888 Plus\n12 GB RAM\n', '6.78 inches (17.22 cm)\n518 PPI, AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nFlash Charging\nUSB Type-C Port\n', '50 + 48 + 12 + 8 MP Quad Primary Cameras\nDual-color LED Flash\n32 MP Front Camera\n'], ['iQOO 9T 5G', 'Octa core (3.2 GHz, Single Core + 2.75 GHz, Tri core + 2 GHz, Quad core)\nSnapdragon 8 Plus Gen 1\n8 GB RAM\n', '6.78 inches (17.22 cm)\n388 PPI, AMOLED\n120 Hz Refresh Rate\n', '4700 mAh\nFlash Charging\nUSB Type-C Port\n', '50 MP + 13 MP + 12 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['OnePlus 10T', 'Octa core (3.2 GHz, Single Core + 2.75 GHz, Tri core + 2 GHz, Quad core)\nSnapdragon 8 Plus Gen 1\n8 GB RAM\n', '6.7 inches (17.02 cm)\n394 PPI, Fluid AMOLED\n120 Hz Refresh Rate\n', '4800 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['realme GT 2 Pro 5G', 'Octa core (3 GHz, Single Core + 2.5 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 8 Gen 1\n8 GB RAM\n', '6.7 inches (17.02 cm)\n526 PPI, AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper Dart Charging\nUSB Type-C Port\n', '50 MP + 50 MP + 2 MP Triple Primary Cameras\nDual LED Flash\n32 MP Front Camera\n'], ['Apple iPhone 13', 'Hexa Core (3.23 GHz, Dual Core + 1.82 GHz, Quad core)\nApple A15 Bionic\n4 GB RAM\n', '6.1 inches (15.49 cm)\n457 PPI, OLED\n60 Hz Refresh Rate\n', '3227 mAh\nFast Charging\nLightning Port\n', '12 MP + 12 MP Dual Primary Cameras\nDual LED Flash\n12 MP Front Camera\n'], ['realme GT Neo 3 5G 150W', 'Octa core (2.85 GHz, Quad Core + 2 GHz, Quad core)\nMediaTek Dimensity 8100\n12 GB RAM\n', '6.7 inches (17.02 cm)\n394 PPI, AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nSuper Dart Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['OnePlus 10R 5G', 'Octa core (2.85 GHz, Quad Core + 2 GHz, Quad core)\nMediaTek Dimensity 8100 Max\n8 GB RAM\n', '6.7 inches (17.02 cm)\n394 PPI, Fluid AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nDual LED Flash\n16 MP Front Camera\n'], ['POCO F4 5G', 'Octa core (3.2 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 870\n6 GB RAM\n', '6.67 inches (16.94 cm)\n395 PPI, AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nSonic Charging\nUSB Type-C Port\n', '64 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n20 MP Front Camera\n'], ['OPPO Reno8 Pro 5G', 'Octa core (2.85 GHz, Quad Core + 2 GHz, Quad core)\nMediaTek Dimensity 8100 Max\n12 GB RAM\n', '6.7 inches (17.02 cm)\n394 PPI, AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nSuper VOOC Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n32 MP Front Camera\n'], ['Asus ROG Phone 5', 'Octa core (2.84 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 888\n8 GB RAM\n', '6.78 inches (17.22 cm)\n395 PPI, AMOLED\n144 Hz Refresh Rate\n', '6000 mAh\nHyper Charging\nUSB Type-C Port\n', '64 MP + 13 MP + 5 MP Triple Primary Cameras\nLED Flash\n24 MP Front Camera\n'], ['iQOO 7 Legend', 'Octa core (2.4 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 888\n8 GB RAM\n', '6.62 inches (16.81 cm)\n398 PPI, AMOLED\n120 Hz Refresh Rate\n', '4000 mAh\nFlash Charging\nUSB Type-C Port\n', '48 MP + 13 MP + 13 MP Triple Primary Cameras\nDual-color LED Flash\n16 MP Front Camera\n'], ['realme GT 5G', 'Octa core (2.42 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 888\n8 GB RAM\n', '6.43 inches (16.33 cm)\n409 PPI, Super AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nSuper Dart Charging\nUSB Type-C Port\n', '64 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['realme 9 Pro Plus', 'Octa core (2.5 GHz, Dual Core + 2 GHz, Hexa Core)\nMediaTek Dimensity 920\n6 GB RAM\n', '6.4 inches (16.26 cm)\n411 PPI, Super AMOLED\n90 Hz Refresh Rate\n', '4500 mAh\nSuper Dart Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['realme GT Neo 2 5G', 'Octa core (3.2 GHz, Single Core + 2.42 GHz, Tri core + 1.8 GHz, Quad core)\nSnapdragon 870\n8 GB RAM\n', '6.62 inches (16.81 cm)\n398 PPI, AMOLED\n120 Hz Refresh Rate\n', '5000 mAh\nSuper Dart Charging\nUSB Type-C Port\n', '64 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['realme X7 Max', 'Octa core (3 GHz, Single Core + 2.6 GHz, Tri core + 2 GHz, Quad core)\nMediaTek Dimensity 1200\n8 GB RAM\n', '6.43 inches (16.33 cm)\n409 PPI, Super AMOLED\n120 Hz Refresh Rate\n', '4500 mAh\nSuper Dart Charging\nUSB Type-C Port\n', '64 MP + 8 MP + 2 MP Triple Primary Cameras\nLED Flash\n16 MP Front Camera\n'], ['OnePlus Nord 2 8GB RAM', 'Octa core (3 GHz, Single Core + 2.6 GHz, Tri core + 2 GHz, Quad core)\nMediaTek Dimensity 1200\n8 GB RAM\n', '6.43 inches (16.33 cm)\n409 PPI, Fluid AMOLED\n90 Hz Refresh Rate\n', '4500 mAh\nWarp Charging\nUSB Type-C Port\n', '50 MP + 8 MP + 2 MP Triple Primary Cameras\nDual LED Flash\n32 MP Front Camera\n']]

In [3]:
print(get_Scores(df,"Samsung Galaxy S23 Ultra 5G"))

[[3.46, 4.71, 4.0, 5.0], [4.05, 4.45, 3.39, 2.15]]


In [4]:
import pandas as pd
import re
# Load the CSV file
data =pd.DataFrame(df, columns=['Phone Name','Performance','Display','Battery','Camera'])


In [5]:
# columns_to_drop = ["Image Link 1", "Image Link 2", "Expert Comment"]
# data = data.drop(columns_to_drop, axis=1)

In [6]:
data.head(1)

,Phone Name,Performance,Display,Battery,Camera
0,Samsung Galaxy S23 Ultra 5G,"Octa core (3.36 GHz, Single Core + 2.8 GHz, Qu...","6.8 inches (17.27 cm)\n501 PPI, Dynamic AMOLED...",5000 mAh\nFast Charging\nUSB Type-C Port\n,200 + 12 + 10 + 10 MP Quad Primary Cameras\nLE...


In [7]:
data.shape

(44, 5)

## Performance

In [8]:
performance_df=data.copy()
columns_to_drop = list(performance_df.columns)

In [9]:
performance_df[['Core', 'Processor', 'RAM']] = performance_df['Performance'].str.split('\n', expand=True).iloc[:,:-1]
performance_df = performance_df.drop(columns_to_drop, axis=1)

In [10]:
performance_df["Processor"].unique()

array(['Snapdragon 8 Gen 2', 'Apple A16 Bionic',
       'MediaTek Dimensity 9200', 'Google Tensor G2',
       'MediaTek Dimensity 9000 Plus', 'Apple A15 Bionic',
       'Snapdragon 8 Plus Gen 1', 'Snapdragon 8 Gen 1',
       'MediaTek Dimensity 9000', 'MediaTek Dimensity 7050',
       'Snapdragon 888', 'Snapdragon 888 Plus', 'MediaTek Dimensity 8100',
       'MediaTek Dimensity 8100 Max', 'Snapdragon 870',
       'MediaTek Dimensity 920', 'MediaTek Dimensity 1200'], dtype=object)

In [11]:
def calculate_phone_score(core, processor, ram):
    # Define a dictionary to assign points based on processors
    processor_scores = {
        'Snapdragon 8 Gen 2':1, 
        'Apple A16 Bionic':8,
       'MediaTek Dimensity 9200':2, 
        'Google Tensor G2':3,
       'MediaTek Dimensity 9000 Plus':2,
        'Apple A15 Bionic':8,
       'Snapdragon 8 Plus Gen 1':5, 
        'Snapdragon 8 Gen 1':4,
       'MediaTek Dimensity 9000':3,
        'MediaTek Dimensity 7050':3,
       'Snapdragon 888':3.5,
        'Snapdragon 888 Plus':4,
        'MediaTek Dimensity 8100':3,
       'MediaTek Dimensity 8100 Max':3.5,
        'Snapdragon 870':3.5,
       'MediaTek Dimensity 920':2,
        'MediaTek Dimensity 1200':2.5,
        # Add more processors and their scores as needed
    }

    # Assign base scores based on core count and RAM size
    core_score = 0
    if 'Octa core' in core:
        core_score = 4
    elif 'Hexa core' in core:
        core_score = 3
    elif 'Quad core' in core:
        core_score = 2
    elif 'Dual core' in core:
        core_score = 1

    ram_score = 0
    if '12 GB' in ram:
        ram_score = 4
    elif '8 GB' in ram:
        ram_score = 3
    elif '6 GB' in ram:
        ram_score = 2
    elif '4 GB' in ram:
        ram_score = 1

    # Get additional processor score from the dictionary
    processor_score = processor_scores.get(processor, 0)

    # Calculate the total score
    total_score = core_score + ram_score + processor_score
    return total_score

# Apply the function to the DataFrame to calculate the scores
data['Performance_Score'] = performance_df.apply(lambda row: calculate_phone_score(row['Core'], row['Processor'], row['RAM']), axis=1)

In [12]:
data['Performance_Score'].unique()

array([ 9. , 12. , 10. , 11. ,  8. , 13. , 11.5, 10.5,  9.5])

## Display

In [13]:
display_df=data.copy()
columns_to_drop = list(display_df.columns)

In [14]:
display_df[['Size', 'Screen_Type', 'Refresh_Rate']]=display_df['Display'].str.split('\n', expand=True).iloc[:,:-1]
display_df = display_df.drop(columns_to_drop, axis=1)

In [15]:
def calculate_display_score(size, screen_type, refresh_rate):
    # Score based on screen size (larger screen size gets higher score)
    size_score = float(size.split()[0])
    
    # Score based on screen type (higher PPI gets higher score, AMOLED screen gets additional points)
    try:
        ppi = float(screen_type.split()[0])
        screen_type_score = ppi / 100  # Scale down PPI to be in a reasonable range
        if 'AMOLED' in screen_type.upper():
            screen_type_score += 5  # Additional points for AMOLED screen
        elif 'OLED' in screen_type.upper():
            screen_type_score += 3     
    except (ValueError, IndexError):
        screen_type_score = 0
    
    # Score based on refresh rate (higher refresh rate gets higher score)
    refresh_rate_score = 0
    if '90 Hz' in refresh_rate:
        refresh_rate_score = 1
    elif '120 Hz' in refresh_rate:
        refresh_rate_score = 2
    elif '144 Hz' in refresh_rate:
        refresh_rate_score = 3    
    
    
    # Calculate the total score
    total_score = size_score + screen_type_score + refresh_rate_score
    return total_score

# Apply the function to the DataFrame to calculate the scores
data['Display_Score'] = display_df.apply(lambda row: calculate_display_score(row['Size'], row['Screen_Type'], row['Refresh_Rate']), axis=1)

In [16]:
data['Display_Score'].unique()

array([18.81, 16.3 , 18.31, 18.94, 16.83, 17.83, 17.5 , 17.32, 16.27,
       18.33, 18.96, 18.25, 16.25, 17.64, 14.27, 17.96, 19.96, 17.66,
       18.32, 13.67, 17.62, 18.73, 17.6 , 17.52, 16.51, 16.52])

## Battery

In [17]:
battery_df=data.copy()
columns_to_drop = list(battery_df.columns)

In [18]:
battery_df

,Phone Name,Performance,Display,Battery,Camera,Performance_Score,Display_Score
0,Samsung Galaxy S23 Ultra 5G,"Octa core (3.36 GHz, Single Core + 2.8 GHz, Qu...","6.8 inches (17.27 cm)\n501 PPI, Dynamic AMOLED...",5000 mAh\nFast Charging\nUSB Type-C Port\n,200 + 12 + 10 + 10 MP Quad Primary Cameras\nLE...,9.0,18.81
1,Apple iPhone 14 Pro Max,"Hexa Core (3.46 GHz, Dual Core + 2.02 GHz, Qua...","6.7 inches (17.02 cm)\n460 PPI, OLED\n120 Hz R...",4323 mAh\nFast Charging\nLightning Port\n,48 MP + 12 MP + 12 MP Triple Primary Cameras\n...,12.0,16.30
2,vivo X90 Pro,"Octa core (3.05 GHz, Single Core + 2.85 GHz, T...","6.78 inches (17.22 cm)\n453 PPI, AMOLED\n120 H...",4870 mAh\nFlash Charging\nUSB Type-C Port\n,50 MP + 12 MP + 50 MP Triple Primary Cameras\n...,10.0,18.31
3,Xiaomi 13 Pro,"Octa core (3.2 GHz, Single Core + 2.8 GHz, Qua...","6.73 inches (17.09 cm)\n521 PPI, AMOLED\n120 H...",4820 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 50 MP + 50 MP Triple Primary Cameras\n...,9.0,18.94
4,Google Pixel 7 Pro 5G,"Octa core (2.85 GHz, Dual Core + 2.35 GHz, Dua...","6.7 inches (17.02 cm)\n513 PPI, OLED\n120 Hz R...",5000 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 48 MP Triple Primary Cameras\n...,11.0,16.83
5,OPPO Find N2 Flip,"Octa core (3.2 GHz, Single Core + 3 GHz, Tri c...","6.8 inches (17.27 cm)\n403 PPI, AMOLED\n120 Hz...",4300 mAh\nSuper VOOC Charging\nUSB Type-C Port\n,50 MP + 8 MP Dual Primary Cameras\nLED Flash\n...,9.0,17.83
6,Samsung Galaxy S23 Plus 5G,"Octa core (3.36 GHz, Single Core + 2.8 GHz, Qu...","6.6 inches (16.76 cm)\n390 PPI, Dynamic AMOLED...",4700 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 10 MP Triple Primary Cameras\n...,8.0,17.50
7,Samsung Galaxy S23 5G,"Octa core (3.36 GHz, Single Core + 2.8 GHz, Qu...","6.1 inches (15.49 cm)\n422 PPI, Dynamic AMOLED...",3900 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 10 MP Triple Primary Cameras\n...,8.0,17.32
8,Apple iPhone 13 Pro Max,"Hexa Core (3.23 GHz, Dual Core + 1.82 GHz, Qua...","6.7 inches (17.02 cm)\n457 PPI, OLED\n120 Hz R...",4352 mAh\nFast Charging\nLightning Port\n,12 MP + 12 MP + 12 MP Triple Primary Cameras\n...,12.0,16.27
9,Samsung Galaxy Z Fold 4 5G,"Octa core (3.18 GHz, Single Core + 2.7 GHz, Tr...","7.6 inches (19.3 cm)\n373 PPI, Dynamic AMOLED\...",4400 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 10 MP Triple Primary Cameras\n...,13.0,18.33


In [19]:
battery_df[['CapacityMah', 'Charge_Type']]=battery_df['Battery'].str.split('\n', expand=True).iloc[:,:-2]
battery_df = battery_df.drop(columns_to_drop, axis=1)
columns_to_drop = list(battery_df.columns)
battery_df[['Capacity','Unit']]=battery_df['CapacityMah'].str.split(' ',expand=True)
battery_df = battery_df.drop(columns_to_drop, axis=1)
battery_df = battery_df.drop(['Unit'], axis=1)
replace_values = lambda x: 1 if 3000 < x <= 3500 else (2 if 3500 < x <= 4000 else (3 if 4000< x <= 4500 else (4 if 4500 < x <= 5000 else 5)))

battery_df['Capacity']=battery_df['Capacity'].astype('int')
data['Battery_Score'] = battery_df['Capacity'].apply(replace_values)

In [20]:
data

,Phone Name,Performance,Display,Battery,Camera,Performance_Score,Display_Score,Battery_Score
0,Samsung Galaxy S23 Ultra 5G,"Octa core (3.36 GHz, Single Core + 2.8 GHz, Qu...","6.8 inches (17.27 cm)\n501 PPI, Dynamic AMOLED...",5000 mAh\nFast Charging\nUSB Type-C Port\n,200 + 12 + 10 + 10 MP Quad Primary Cameras\nLE...,9.0,18.81,4
1,Apple iPhone 14 Pro Max,"Hexa Core (3.46 GHz, Dual Core + 2.02 GHz, Qua...","6.7 inches (17.02 cm)\n460 PPI, OLED\n120 Hz R...",4323 mAh\nFast Charging\nLightning Port\n,48 MP + 12 MP + 12 MP Triple Primary Cameras\n...,12.0,16.30,3
2,vivo X90 Pro,"Octa core (3.05 GHz, Single Core + 2.85 GHz, T...","6.78 inches (17.22 cm)\n453 PPI, AMOLED\n120 H...",4870 mAh\nFlash Charging\nUSB Type-C Port\n,50 MP + 12 MP + 50 MP Triple Primary Cameras\n...,10.0,18.31,4
3,Xiaomi 13 Pro,"Octa core (3.2 GHz, Single Core + 2.8 GHz, Qua...","6.73 inches (17.09 cm)\n521 PPI, AMOLED\n120 H...",4820 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 50 MP + 50 MP Triple Primary Cameras\n...,9.0,18.94,4
4,Google Pixel 7 Pro 5G,"Octa core (2.85 GHz, Dual Core + 2.35 GHz, Dua...","6.7 inches (17.02 cm)\n513 PPI, OLED\n120 Hz R...",5000 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 48 MP Triple Primary Cameras\n...,11.0,16.83,4
5,OPPO Find N2 Flip,"Octa core (3.2 GHz, Single Core + 3 GHz, Tri c...","6.8 inches (17.27 cm)\n403 PPI, AMOLED\n120 Hz...",4300 mAh\nSuper VOOC Charging\nUSB Type-C Port\n,50 MP + 8 MP Dual Primary Cameras\nLED Flash\n...,9.0,17.83,3
6,Samsung Galaxy S23 Plus 5G,"Octa core (3.36 GHz, Single Core + 2.8 GHz, Qu...","6.6 inches (16.76 cm)\n390 PPI, Dynamic AMOLED...",4700 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 10 MP Triple Primary Cameras\n...,8.0,17.50,4
7,Samsung Galaxy S23 5G,"Octa core (3.36 GHz, Single Core + 2.8 GHz, Qu...","6.1 inches (15.49 cm)\n422 PPI, Dynamic AMOLED...",3900 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 10 MP Triple Primary Cameras\n...,8.0,17.32,2
8,Apple iPhone 13 Pro Max,"Hexa Core (3.23 GHz, Dual Core + 1.82 GHz, Qua...","6.7 inches (17.02 cm)\n457 PPI, OLED\n120 Hz R...",4352 mAh\nFast Charging\nLightning Port\n,12 MP + 12 MP + 12 MP Triple Primary Cameras\n...,12.0,16.27,3
9,Samsung Galaxy Z Fold 4 5G,"Octa core (3.18 GHz, Single Core + 2.7 GHz, Tr...","7.6 inches (19.3 cm)\n373 PPI, Dynamic AMOLED\...",4400 mAh\nFast Charging\nUSB Type-C Port\n,50 MP + 12 MP + 10 MP Triple Primary Cameras\n...,13.0,18.33,3


## Camera

In [21]:
camera_df=data.copy()
columns_to_drop = list(camera_df.columns)
camera_df[['Back','Flash','Front']] =camera_df['Camera'].str.split('\n', expand=True).iloc[:,:-1]
camera_df = camera_df.drop(columns_to_drop, axis=1)
camera_df = camera_df.drop(['Flash'], axis=1)
def sum_of_digits(s):
    # Extract digits using regular expression and sum them up
    digits = [int(digit) for digit in re.findall(r'\d+', s)]
    return sum(digits)

# Apply the function to each row using apply() along axis=1
data['Camera_score'] = camera_df.apply(lambda row: sum_of_digits(row['Back'] + row['Front']), axis=1)

## Normalization

In [ ]:
data['Performance_Score']=round((5*data['Performance_Score']/max(data['Performance_Score'])),2)
data['Display_Score']=round((5*data['Display_Score']/max(data['Display_Score'])),2)
data['Battery_Score']=round((5*data['Battery_Score']/max(data['Battery_Score'])),2)
data['Camera_score']=round((5*data['Camera_score']/max(data['Camera_score'])),2)

In [ ]:
data = data.drop(['Performance','Display','Battery','Camera'], axis=1)

## Phone Information

In [ ]:
Averages=[data['Performance_Score'].mean(),data['Display_Score'].mean(),data['Battery_Score'].mean(),data['Camera_score'].mean()]

In [ ]:
Averages = list(map(lambda x: round(x, 2),Averages))

In [ ]:
Averages

In [ ]:
def get_phone_info(phone_name):
    row = data[data['Phone Name'] == phone_name].iloc[0]
    phone_info = row.values.tolist()
    return phone_info

# Example usage:
phone_name = "Samsung Galaxy S23 Ultra 5G"
phone_info = get_phone_info(phone_name)
print(phone_info)

In [ ]:
phone_info=phone_info[1:]

In [ ]:
phoneScore=[]
phoneScore.append(phone_info)
phoneScore.append(Averages)

In [ ]:
phoneScore